In [ ]:
%config Completer.use_jedi = False

In [1]:
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mne
import json
import scipy.stats
import scipy.signal as sig

# import scipy.io as sio
# from scipy import signal

import pac

import simple_pipeline


suffix = '_1_200_double'#'_1ch_nv'
gamma = [ 1, 200]
beta  = [ 1, 50]

filt_wind = np.ones((3, 3))
filt_wind /= filt_wind.sum()

# functions

In [2]:
with open('config.json') as f:
    config = json.load(f)
    channels = config['channels']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
linestyles = ['-', ':', '--']

groups = ['PD Med Off', 'PD Med On', 'CTL']
event_types = ['Target', 'Standard', 'Novelty']

In [3]:
def plot_pac(pac, high_freq=gamma, low_freq=beta, ax=None, interpolation='None', **kwargs):
    if ax is None:
        fig = plt.figure(figsize=(7, 15))
        ax = fig.subplots()

    im = ax.imshow((pac), origin='lower', interpolation=interpolation, #'nearest', 
                   extent=low_freq+high_freq,
#                    aspect='auto', )
                   aspect=np.diff(low_freq)/np.diff(high_freq), **kwargs)

    if ax is None:
        plt.show()
        
    return im

In [4]:
def get_percent(arr, thr=0.95):
    if arr.ndim > 1:
        arr = arr.ravel()
    freq, bins = np.histogram(arr, bins=100)
    return (
        bins[:-1][(freq / freq.sum()).cumsum() > thr][0], 
        bins[1:][(freq / freq.sum()).cumsum() > thr][0]
    )

In [5]:
def save_fig(path):
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    plt.savefig(path)

# Create Task list in `tasks_df`

In [6]:
if __name__ == '__main__':
    tasks_df = simple_pipeline.create_tasks_df()
    
    completed = []
    for task in tasks_df.iloc:
        json_path = os.path.join(task['dir'], task['file_formatter'].format(f'completed{suffix}.json'))
        completed.append(os.path.exists(json_path))

    tasks_df = pd.concat([tasks_df, pd.DataFrame({'completed': completed})], axis=1)

In [7]:
os.path.join(task['dir'], task['file_formatter'].format(f'mvls{suffix}.npz'))

'/home/kiani/DS/ds003490-download/sub-050/ses-01/eeg/sub-050_ses-01_task-Rest_mvls_1_200_double.npz'

# Load Data

In [ ]:
# MVL

mvl_2ds_time = [[] for k in groups] # np.zeros((3, 64, 169, 37 * 12))
mvl_2ds = [[] for k in groups] # np.zeros((3, 64, 61, 13 )//169, 37))
mvls = [[] for k in groups] # np.zeros((3, 64))

for task in tasks_df.iloc:
# if 1:
    task_mvls = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvls{suffix}.npz')))
    task_mvl_2ds = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2ds{suffix}.npz')))
    
    # mvls
    nbchan = task_mvls[task_mvls.files[0]].shape[0]
    mvl = np.zeros((3, nbchan))
    for i, event_type in enumerate(sorted(task_mvls.files)):
        mvl[i] = task_mvls[event_type].diagonal()
        
    mvls[task.pd_drug_type].append(mvl)
    
    # mvl_2ds
    mvl_2d = np.zeros((3, nbchan, gamma[1] - gamma[0] + 1, beta[1] - beta[0] + 1))
    for i, event_type in enumerate(sorted(task_mvl_2ds.files)):
        mvl_2d[i] = task_mvl_2ds[event_type].diagonal(0, 0, 1).transpose((2, 0, 1))
        
    mvl_2ds[task.pd_drug_type].append(mvl_2d)

mvls = np.array(mvls)              # --> (pd_drug_type, subjects, event_types, channels)
mvl_2ds = np.array(mvl_2ds)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs)
mvl_2ds_time = np.array(mvl_2ds_time)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs * 12)

In [8]:
with open('config.json') as f:
    config = json.load(f)
base_path = config['BASE_PATH']
ds_path = os.path.join(base_path, 'ds003490-download')
nbchan = len(config['channels']) - 1

In [12]:
# MVL 2Ds

mvl_2ds_mmap_path = os.path.join(ds_path, f'MVL_2ds_{suffix}.mmap')

mvl_2ds = np.memmap(mvl_2ds_mmap_path, dtype=float, mode='w+',
                    shape=(3,  # pd_drug_type
                           25, # Subjects
                           3,  # event_types
                           nbchan, # Channels
                           gamma[1] - gamma[0] + 1, # High Freq
                           beta[1] - beta[0] + 1, # Low Freq
                          ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_mvl_2ds = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2ds{suffix}.npz')))
    for i, event_type in enumerate(sorted(task_mvl_2ds.files)):
        mvl_2ds[task.pd_drug_type, subs[task.pd_drug_type], i] = \
            task_mvl_2ds[event_type].diagonal(0, 0, 1).transpose((2, 0, 1))
    
        
    subs[task.pd_drug_type] += 1

# mvl_2ds = np.array(mvl_2ds)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs)
# mvl_2ds_time = np.array(mvl_2ds_time)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs * 12)

[0, 0, 0] 1
[0, 1, 0] 0
[1, 1, 0] 1
[1, 2, 0] 0
[2, 2, 0] 2
[2, 2, 1] 0
[3, 2, 1] 1
[3, 3, 1] 2
[3, 3, 2] 0
[4, 3, 2] 1
[4, 4, 2] 0
[5, 4, 2] 1
[5, 5, 2] 1
[5, 6, 2] 0
[6, 6, 2] 1
[6, 7, 2] 0
[7, 7, 2] 1
[7, 8, 2] 0
[8, 8, 2] 0
[9, 8, 2] 1
[9, 9, 2] 1
[9, 10, 2] 0
[10, 10, 2] 1
[10, 11, 2] 0
[11, 11, 2] 0
[12, 11, 2] 1
[12, 12, 2] 0
[13, 12, 2] 1
[13, 13, 2] 1
[13, 14, 2] 0
[14, 14, 2] 0
[15, 14, 2] 1
[15, 15, 2] 1
[15, 16, 2] 0
[16, 16, 2] 1
[16, 17, 2] 0
[17, 17, 2] 1
[17, 18, 2] 0
[18, 18, 2] 0
[19, 18, 2] 1
[19, 19, 2] 0
[20, 19, 2] 1
[20, 20, 2] 1
[20, 21, 2] 0
[21, 21, 2] 1
[21, 22, 2] 0
[22, 22, 2] 0
[23, 22, 2] 1
[23, 23, 2] 0
[24, 23, 2] 1
[24, 24, 2] 0
[25, 24, 2] 1
[25, 25, 2] 2
[25, 25, 3] 2
[25, 25, 4] 2
[25, 25, 5] 2
[25, 25, 6] 2
[25, 25, 7] 2
[25, 25, 8] 2
[25, 25, 9] 2
[25, 25, 10] 2
[25, 25, 11] 2
[25, 25, 12] 2
[25, 25, 13] 2
[25, 25, 14] 2
[25, 25, 15] 2
[25, 25, 16] 2
[25, 25, 17] 2
[25, 25, 18] 2
[25, 25, 19] 2
[25, 25, 20] 2
[25, 25, 21] 2
[25, 25, 22] 2
[25, 25,

In [ ]:
mvl_time_mmap_path = os.path.join(ds_path, f'PAC_time_{suffix}.mmap')
mvl_time = np.memmap(mvl_time_mmap_path, dtype=float, mode='w+',
                     shape=(3,  # pd_drug_type
                            25, # Subjects
                            3,  # event_types
                            nbchan, # Channels
                            gamma[1] - gamma[0] + 1, # High Freq
                            beta[1] - beta[0] + 1, # Low Freq
                            6, # Time sections
                           ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_mvl_2ds_time = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2d_times{suffix}.npz')))
    
    # mvl_2ds
    for i, event_type in enumerate(sorted(task_mvl_2ds_time.files)):
        mvl_time[task.pd_drug_type, subs[task.pd_drug_type], i] = \
            task_mvl_2ds_time[event_type]
    
#         mvl_2d_time[i] = task_mvl_2ds_time[event_type]
    subs[task.pd_drug_type] += 1
        
#     mvl_2ds_time[task.pd_drug_type].append(mvl_2d_time)

[0, 0, 0] 1
[0, 1, 0] 0
[1, 1, 0] 1
[1, 2, 0] 0
[2, 2, 0] 2
[2, 2, 1] 0
[3, 2, 1] 1
[3, 3, 1] 2
[3, 3, 2] 0
[4, 3, 2] 1
[4, 4, 2] 0
[5, 4, 2] 1
[5, 5, 2] 1
[5, 6, 2] 0
[6, 6, 2] 1
[6, 7, 2] 0
[7, 7, 2] 1
[7, 8, 2] 0
[8, 8, 2] 0
[9, 8, 2] 1
[9, 9, 2] 1
[9, 10, 2] 0
[10, 10, 2] 1
[10, 11, 2] 0
[11, 11, 2] 0
[12, 11, 2] 1
[12, 12, 2] 0
[13, 12, 2] 1
[13, 13, 2] 1
[13, 14, 2] 0
[14, 14, 2] 0
[15, 14, 2] 1
[15, 15, 2] 1
[15, 16, 2] 0
[16, 16, 2] 1
[16, 17, 2] 0
[17, 17, 2] 1
[17, 18, 2] 0
[18, 18, 2] 0
[19, 18, 2] 1
[19, 19, 2] 0
[20, 19, 2] 1
[20, 20, 2] 1
[20, 21, 2] 0
[21, 21, 2] 1
[21, 22, 2] 0
[22, 22, 2] 0
[23, 22, 2] 1
[23, 23, 2] 0
[24, 23, 2] 1
[24, 24, 2] 0
[25, 24, 2] 1
[25, 25, 2] 2
[25, 25, 3] 2
[25, 25, 4] 2
[25, 25, 5] 2
[25, 25, 6] 2
[25, 25, 7] 2
[25, 25, 8] 2
[25, 25, 9] 2
[25, 25, 10] 2
[25, 25, 11] 2
[25, 25, 12] 2
[25, 25, 13] 2
[25, 25, 14] 2
[25, 25, 15] 2
[25, 25, 16] 2
[25, 25, 17] 2
[25, 25, 18] 2
[25, 25, 19] 2
[25, 25, 20] 2
[25, 25, 21] 2
[25, 25, 22] 2
[25, 25,

In [12]:
# ERP

nbtime = 601

erp_mmap_path = os.path.join(ds_path, f'ERPs_{suffix}.mmap')
epochs = np.memmap(erp_mmap_path, dtype=float, mode='w+',
                     shape=(3,  # pd_drug_type
                            25, # Subjects
                            3,  # event_types
                            2,  # (mean, std)
                            nbchan, # Channels
                            nbtime, # time
                           ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_epochs = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'epochs{suffix}.npz')))
    
    # epochs
    for i, event_type in enumerate(sorted(task_epochs.files)):
        print(subs, task.pd_drug_type)
        epochs[task.pd_drug_type, subs[task.pd_drug_type], i, 0] = task_epochs[event_type].mean(axis=0)
        epochs[task.pd_drug_type, subs[task.pd_drug_type], i, 1] = task_epochs[event_type].std(axis=0)
        
    # epochs[task.pd_drug_type].append(epoch)
    subs[task.pd_drug_type] += 1

[0, 0, 0] 1
[0, 0, 0] 1
[0, 0, 0] 1
[0, 0, 0] 1
[0, 1, 0] 0
[0, 1, 0] 0
[0, 1, 0] 0
[0, 1, 0] 0
[1, 1, 0] 1
[1, 1, 0] 1
[1, 1, 0] 1
[1, 1, 0] 1
[1, 2, 0] 0
[1, 2, 0] 0
[1, 2, 0] 0
[1, 2, 0] 0
[2, 2, 0] 2
[2, 2, 0] 2
[2, 2, 0] 2
[2, 2, 0] 2
[2, 2, 1] 0
[2, 2, 1] 0
[2, 2, 1] 0
[2, 2, 1] 0
[3, 2, 1] 1
[3, 2, 1] 1
[3, 2, 1] 1
[3, 2, 1] 1
[3, 3, 1] 2
[3, 3, 1] 2
[3, 3, 1] 2
[3, 3, 1] 2
[3, 3, 2] 0
[3, 3, 2] 0
[3, 3, 2] 0
[3, 3, 2] 0
[4, 3, 2] 1
[4, 3, 2] 1
[4, 3, 2] 1
[4, 3, 2] 1
[4, 4, 2] 0
[4, 4, 2] 0
[4, 4, 2] 0
[4, 4, 2] 0
[5, 4, 2] 1
[5, 4, 2] 1
[5, 4, 2] 1
[5, 4, 2] 1
[5, 5, 2] 1
[5, 5, 2] 1
[5, 5, 2] 1
[5, 5, 2] 1
[5, 6, 2] 0
[5, 6, 2] 0
[5, 6, 2] 0
[5, 6, 2] 0
[6, 6, 2] 1
[6, 6, 2] 1
[6, 6, 2] 1
[6, 6, 2] 1
[6, 7, 2] 0
[6, 7, 2] 0
[6, 7, 2] 0
[6, 7, 2] 0
[7, 7, 2] 1
[7, 7, 2] 1
[7, 7, 2] 1
[7, 7, 2] 1
[7, 8, 2] 0
[7, 8, 2] 0
[7, 8, 2] 0
[7, 8, 2] 0
[8, 8, 2] 0
[8, 8, 2] 0
[8, 8, 2] 0
[8, 8, 2] 0
[9, 8, 2] 1
[9, 8, 2] 1
[9, 8, 2] 1
[9, 8, 2] 1
[9, 9, 2] 1
[9, 9, 2] 1
[9, 9, 2] 1
[9, 

True

In [26]:
# MVL 2Ds Cross

selected_channels = ['FC3','FC4','AF3','AF4','F3','F4','Fz','Pz','Cz','FCz']
selected_channels_index = [38, 57, 33, 61, 2, 29, 1, 12, 23, 39]
if [channels[i] for i in selected_channels_index] == selected_channels_index:
    print('Correct indices has been selected')

mvl_cross_mmap_path = os.path.join(ds_path, f'MVL_cross_{suffix}.mmap')

mvl_cross = np.memmap(mvl_cross_mmap_path, dtype=float, mode='w+',
                      shape=(3,  # pd_drug_type
                             25, # Subjects
                             3,  # event_types
                             len(selected_channels), # Selected Channels
                             len(selected_channels), # Selected Channels
                             gamma[1] - gamma[0] + 1, # High Freq
                             beta[1] - beta[0] + 1, # Low Freq
                            ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_mvl_2ds = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_2ds{suffix}.npz')))
    for i, event_type in enumerate(sorted(task_mvl_2ds.files)):
        mvl_cross[task.pd_drug_type, subs[task.pd_drug_type], i] = \
            task_mvl_2ds[event_type][selected_channels_index][:, selected_channels_index]
        
    subs[task.pd_drug_type] += 1

# mvl_2ds = np.array(mvl_2ds)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs)
# mvl_2ds_time = np.array(mvl_2ds_time)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs * 12)

[0, 0, 0] 1
[0, 1, 0] 0
[1, 1, 0] 1
[1, 2, 0] 0
[2, 2, 0] 2
[2, 2, 1] 0
[3, 2, 1] 1
[3, 3, 1] 2
[3, 3, 2] 0
[4, 3, 2] 1
[4, 4, 2] 0
[5, 4, 2] 1
[5, 5, 2] 1
[5, 6, 2] 0
[6, 6, 2] 1
[6, 7, 2] 0
[7, 7, 2] 1
[7, 8, 2] 0
[8, 8, 2] 0
[9, 8, 2] 1
[9, 9, 2] 1
[9, 10, 2] 0
[10, 10, 2] 1
[10, 11, 2] 0
[11, 11, 2] 0
[12, 11, 2] 1
[12, 12, 2] 0
[13, 12, 2] 1
[13, 13, 2] 1
[13, 14, 2] 0
[14, 14, 2] 0
[15, 14, 2] 1
[15, 15, 2] 1
[15, 16, 2] 0
[16, 16, 2] 1
[16, 17, 2] 0
[17, 17, 2] 1
[17, 18, 2] 0
[18, 18, 2] 0
[19, 18, 2] 1
[19, 19, 2] 0
[20, 19, 2] 1
[20, 20, 2] 1
[20, 21, 2] 0
[21, 21, 2] 1
[21, 22, 2] 0
[22, 22, 2] 0
[23, 22, 2] 1
[23, 23, 2] 0
[24, 23, 2] 1
[24, 24, 2] 0
[25, 24, 2] 1
[25, 25, 2] 2
[25, 25, 3] 2
[25, 25, 4] 2
[25, 25, 5] 2
[25, 25, 6] 2
[25, 25, 7] 2
[25, 25, 8] 2
[25, 25, 9] 2
[25, 25, 10] 2
[25, 25, 11] 2
[25, 25, 12] 2
[25, 25, 13] 2
[25, 25, 14] 2
[25, 25, 15] 2
[25, 25, 16] 2
[25, 25, 17] 2
[25, 25, 18] 2
[25, 25, 19] 2
[25, 25, 20] 2
[25, 25, 21] 2
[25, 25, 22] 2
[25, 25,

In [9]:
# MVL 2Ds Cross

selected_channels = ['FC3','FC4','AF3','AF4','F3','F4','Fz','Pz','Cz','FCz']
selected_channels_index = [38, 57, 33, 61, 2, 29, 1, 12, 23, 39]
if [channels[i] for i in selected_channels_index] == selected_channels_index:
    print('Correct indices has been selected')

mvl_cross_time_mmap_path = os.path.join(ds_path, f'MVL_cross_time_{suffix}.mmap')

mvl_cross_time = np.memmap(mvl_cross_time_mmap_path, dtype=float, mode='w+',
                           shape=(3,  # pd_drug_type
                                  25, # Subjects
                                  3,  # event_types
                                  len(selected_channels), # Selected Channels
                                  len(selected_channels), # Selected Channels
                                  gamma[1] - gamma[0] + 1, # High Freq
                                  beta[1] - beta[0] + 1, # Low Freq
                                  6
                                 ))

subs = [0, 0, 0]
for task in tasks_df.iloc:
    print(subs, task.pd_drug_type)
    task_mvl_2ds = np.load(os.path.join(task['dir'], task['file_formatter'].format(f'mvl_cross_times{suffix}.npz')))
    for i, event_type in enumerate(sorted(task_mvl_2ds.files)):
        mvl_cross_time[task.pd_drug_type, subs[task.pd_drug_type], i] = \
            task_mvl_2ds[event_type]
        
    subs[task.pd_drug_type] += 1

# mvl_2ds = np.array(mvl_2ds)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs)
# mvl_2ds_time = np.array(mvl_2ds_time)        # --> (pd_drug_type, subjects, event_types, channels, high_freqs, low_freqs * 12)

[0, 0, 0] 1
[0, 1, 0] 0
[1, 1, 0] 1
[1, 2, 0] 0
[2, 2, 0] 2
[2, 2, 1] 0
[3, 2, 1] 1
[3, 3, 1] 2
[3, 3, 2] 0
[4, 3, 2] 1
[4, 4, 2] 0
[5, 4, 2] 1
[5, 5, 2] 1
[5, 6, 2] 0
[6, 6, 2] 1
[6, 7, 2] 0
[7, 7, 2] 1
[7, 8, 2] 0
[8, 8, 2] 0
[9, 8, 2] 1
[9, 9, 2] 1
[9, 10, 2] 0
[10, 10, 2] 1
[10, 11, 2] 0
[11, 11, 2] 0
[12, 11, 2] 1
[12, 12, 2] 0
[13, 12, 2] 1
[13, 13, 2] 1
[13, 14, 2] 0
[14, 14, 2] 0
[15, 14, 2] 1
[15, 15, 2] 1
[15, 16, 2] 0
[16, 16, 2] 1
[16, 17, 2] 0
[17, 17, 2] 1
[17, 18, 2] 0
[18, 18, 2] 0
[19, 18, 2] 1
[19, 19, 2] 0
[20, 19, 2] 1
[20, 20, 2] 1
[20, 21, 2] 0
[21, 21, 2] 1
[21, 22, 2] 0
[22, 22, 2] 0
[23, 22, 2] 1
[23, 23, 2] 0
[24, 23, 2] 1
[24, 24, 2] 0
[25, 24, 2] 1
[25, 25, 2] 2
[25, 25, 3] 2
[25, 25, 4] 2
[25, 25, 5] 2
[25, 25, 6] 2
[25, 25, 7] 2
[25, 25, 8] 2
[25, 25, 9] 2
[25, 25, 10] 2
[25, 25, 11] 2
[25, 25, 12] 2
[25, 25, 13] 2
[25, 25, 14] 2
[25, 25, 15] 2
[25, 25, 16] 2
[25, 25, 17] 2
[25, 25, 18] 2
[25, 25, 19] 2
[25, 25, 20] 2
[25, 25, 21] 2
[25, 25, 22] 2
[25, 25,

In [10]:
selected_channels = ['FC3', 'FC4', 'AF3',
                     'AF4', 'F3', 'F4', 'Fz', 'Pz', 'Cz', 'FCz']
selected_channels_index = [38, 57, 33, 61, 2, 29, 1, 12, 23, 39]
if [channels[i] for i in selected_channels_index] == selected_channels_index:
    print('Correct indices has been selected')

mvl_cross_time_mmap_path = os.path.join(ds_path, f'MVL_cross_time_{suffix}.mmap')

mvl_cross_time = np.memmap(mvl_cross_time_mmap_path, dtype=float,
                           shape=(3,  # pd_drug_type
                                  25, # Subjects
                                  3,  # event_types
                                  len(selected_channels), # Selected Channels
                                  len(selected_channels), # Selected Channels
                                  gamma[1] - gamma[0] + 1, # High Freq
                                  beta[1] - beta[0] + 1, # Low Freq
                                  6
                                 ))

mx = mvl_cross_time.max()

In [15]:
nrm = np.linalg.norm(mvl_cross_time)

In [16]:
nrm

8344569328054774.0

In [16]:
[f for f in os.listdir(ds_path) if f.endswith('mmap')]

['ERPs__1_200_double.mmap',
 'MVL_cross__1_200_double.mmap',
 'MVL_cross_time__1_200_double.mmap',
 'MVL_2ds__1_200_double.mmap',
 'PAC_time__1_200_double.mmap']

In [15]:
!ls -sh /home/kiani/DS/ds003490-download/*.mmap

4.0K /home/kiani/DS/ds003490-download/ERPs__1_200_double.mmap
1.1G /home/kiani/DS/ds003490-download/MVL_2ds__1_200_double.mmap
1.7G /home/kiani/DS/ds003490-download/MVL_cross__1_200_double.mmap
 11G /home/kiani/DS/ds003490-download/MVL_cross_time__1_200_double.mmap
6.4G /home/kiani/DS/ds003490-download/PAC_time__1_200_double.mmap


In [13]:
task_mvl_2ds[event_type][selected_channels_index][:, selected_channels_index]

(10, 10, 200, 50, 6)

In [10]:
selected_channels = ['FC3','FC4','AF3','AF4','F3','F4','Fz','Pz','Cz','FCz']
selected_channels_index = [38, 57, 33, 61, 2, 29, 1, 12, 23, 39]
if [channels[i] for i in selected_channels_index] == selected_channels_index:
    print('Correct indices has been selected')

mvl_cross_time_mmap_path = os.path.join(ds_path, f'MVL_cross_time_{suffix}.mmap')

mvl_cross_time = np.memmap(mvl_cross_time_mmap_path, dtype=float, mode='r',
                           shape=(3,  # pd_drug_type
                                  25, # Subjects
                                  3,  # event_types
                                  len(selected_channels), # Selected Channels
                                  len(selected_channels), # Selected Channels
                                  gamma[1] - gamma[0] + 1, # High Freq
                                  beta[1] - beta[0] + 1, # Low Freq
                                  6
                                 ))
np.linalg.norm(mvl_cross_time)

8344569328054774.0